Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0

# Visualize the dynamic graph using Networkx

This notebook provides visualizes the dynamic graph using Network X. The nodes are customers and merchants, and the edges are transactions between them.

## Table of Contents
1. Load processed node and edge data
2. Load node attributes from raw data
3. Build graph with networkx and visualize a specific snapshot


In [ ]:
import sys 
sys.path.append('../../src/anomaly_detection_spatial_temporal_data/')

In [ ]:
import pandas as pd
import numpy as np
from visualization.dynamic_graph import DynamicHomoGraph

In [ ]:
#rerun this cell again if your plot looks very small
from matplotlib import pyplot as plt
import matplotlib

font = {'family': 'normal', 'size': 18}
matplotlib.rc('font', **font)

import matplotlib.cm as cm
plt.rcParams["figure.figsize"] = (30,18)

## Load node and edge data from 1.1 processing notebook

In [ ]:
edge_list_w_label_filepath = '../../data/02_intermediate/financial_fraud/edge_list.npz'
node_id_mapping_filepath = '../../data/02_intermediate/financial_fraud/node_id.csv'

In [ ]:
edge_list = np.load(edge_list_w_label_filepath)

node_id = pd.read_csv(node_id_mapping_filepath)

## Load node attribute value 

The customer/merchant IDs are used to color the nodes in the visualization.

In [ ]:
raw_data_path = '../../data/01_raw/financial_fraud/bs140513_032310.csv'

raw_trans_data = pd.read_csv(raw_data_path)

In [ ]:
customer_attribute_data = raw_trans_data[['step','customer','age','gender','zipcodeOri']]
merchant_attribute_data = raw_trans_data[['step','merchant','zipMerchant','category']]

In [ ]:
# customer_attribute_data.head(10)

In [ ]:
# merchant_attribute_data.head(10)

In [ ]:
customer_attribute_data_deduped = customer_attribute_data.drop_duplicates(subset=['customer'], keep='last', )
merchant_attribute_data_deduped = merchant_attribute_data.drop_duplicates(subset=['merchant'], keep='last', )

In [ ]:
# customer_attribute_data.shape, customer_attribute_data_deduped.shape

In [ ]:
# merchant_attribute_data.shape, merchant_attribute_data_deduped.shape

## Build graph with networkx and visualize a specific snapshot

In [ ]:
dynamic_payment_network = DynamicHomoGraph(
    node_id, edge_list['data'], 
    src_node_attribute=customer_attribute_data_deduped, 
    dst_node_attribute=merchant_attribute_data_deduped, 
    load_time_steps=10
)

In [ ]:
# dynamic_payment_network._determine_object_type(4112)

In [ ]:
dynamic_payment_network.draw_nx_graph_w_edge_label_at_specific_time(1)

#### check some edge label

In [ ]:
edge_list['data'][np.where((edge_list['data'][:,0]==1444)&(edge_list['data'][:,1]==4138))[0], :],edge_list['data'][np.where((edge_list['data'][:,0]==3358)&(edge_list['data'][:,1]==4138))[0], :]

#### check node features

In [ ]:
dynamic_payment_network.src_node_static_attribute[1444],dynamic_payment_network.dst_node_static_attribute[4138]

In [ ]:
# plot a different snapshot
dynamic_payment_network.draw_nx_graph_w_edge_label_at_specific_time(2)